In [1]:
using Distributions
using Statistics
using DataFrames
using CSV #change to DelimitedFiles usage later

In [2]:
frip = DataFrame(CSV.File("clean_fRIP.txt"))
redc = DataFrame(CSV.File("K562.pvalue.tab"))[:, ["gene_name_un","gene_type","zscore", "chP"]]
final = innerjoin(frip, redc, on = :RNA => :gene_name_un)
@view final[1:10, [1,26,27,28]]

,RNA,gene_type,zscore,chP
,String31,String,Float64,Float64
1,ZMIZ1,protein_coding,1.55229,1.99568
2,MFSD14B,protein_coding,-0.0219071,0.253634
3,MTR,protein_coding,0.262814,0.568714
4,TIRAP,protein_coding,0.188857,0.48687
5,LMBR1,protein_coding,0.931878,1.30912
6,STEAP1B,protein_coding,-1.50231,-1.38462
7,KDM7A,protein_coding,0.272723,0.579679
8,LINC01811,lncRNA,0.557746,0.895093
9,ZNF318,protein_coding,-0.843082,-0.655099


In [3]:
function get_thr_gamma(sample, scl, shp)
    s = sort(sample, rev = true)
    thr2pk = Dict{Float64, Float64}()
    for i in range(1, length(s))
        pki = 1 - cdf(Gamma(scl, shp), s[i])
        qki = 1 - pki
        p = 0
        for j in range(i, length(s))
            addition = pdf(Binomial(length(s), pki), j)
            p += addition
        end
        thr2pk[s[i]] = p
    end
    return thr2pk
end

function prot_threshold(name)
    vect = final[(final[!, name] .< 1) .& (final.gene_type .== "protein_coding"), :][!, name]
    scl, shp = var(vect)/mean(vect), mean(vect)^2/var(vect)
    vect = final[(final[!, name] .< 10), :][!, name]
    cur_thr = get_thr_gamma(vect, scl ,shp)
    return sort(DataFrame(thr = collect(keys(cur_thr)), p = collect(values(cur_thr))), 
        [:p, order(:thr, rev = true)])[1,1]
end

prot_threshold (generic function with 1 method)

In [4]:
#collect protein thresholds
prot_thr = Dict{String, Float64}()
for name in names(final[:, 2:25])
    prot_thr[name] = prot_threshold(name)
end
prot_thr

Dict{String, Float64} with 24 entries:
  "rbbp5"  => 1.44599
  "suz12"  => 1.34986
  "cbx3"   => 1.65439
  "wdr5"   => 1.39616
  "lsd1"   => 0.976197
  "imp1"   => 1.16915
  "dnmt1"  => 1.51324
  "hnrnph" => 3.08671
  "hnrnpu" => 2.61515
  "phf8"   => 1.54915
  "nup98"  => 1.16873
  "chd7"   => 1.18982
  "pcaf"   => 1.08796
  "pcbx"   => 1.32141
  "chd4"   => 1.14525
  "stag2"  => 1.69785
  "adar"   => 7.22305
  "hur"    => 1.54382
  "hdac1"  => 1.74062
  "ctcf"   => 1.32479
  "ezh2"   => 1.45841
  "chd1"   => 1.22795
  "cbp"    => 1.22172
  "pabp"   => 1.02727

In [7]:
# collect zscore threshold
bg_zscore = final[(final.gene_type .== "protein_coding") .& (final.zscore .< 3), :].zscore
test = map(x -> x + 3, bg_zscore)
test = test[test .> 0]
scl, shp = var(test)/mean(test), mean(test)^2/var(test)
zscr_gshift = fit(Gamma, test[test .!= 0])
t2 = get_thr_gamma(final.zscore, scl, shp)
zs_thr = sort(DataFrame(thr = collect(keys(t2)), p = collect(values(t2))), 
        [:p, order(:thr, rev = true)])[1,1]

1.09531969444167